In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import tensorflow as tf
import numpy as np
import PIL
#import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Sequential
from tensorflow.compat.v1 import ConfigProto

config = ConfigProto()
config.gpu_options.allow_growth = True
tf.__version__

'2.3.0'

In [2]:
data_train, data_test = tf.keras.datasets.cifar10.load_data()
images_train, labels_train = data_train
images_test, labels_test = data_test
# label to_categorical
labels_train = tf.keras.utils.to_categorical(labels_train)
labels_test = tf.keras.utils.to_categorical(labels_test)

#images_train = images_train.astype('float32')
#images_train = images_train/255.0
#images_test = images_test.astype('float32')
#images_test = images_test/255.0

In [3]:
images_train.shape, labels_train.shape

((50000, 32, 32, 3), (50000, 10))

In [4]:
# validation split & expand_dims
validation_size = 10000
images_validation = images_train[-validation_size:]
images_train = images_train[:validation_size]
#images_train = np.expand_dims(images_train, axis=-1)
labels_validation = labels_train[-validation_size:]
labels_train = labels_train[:validation_size]
#labels_train = np.expand_dims(labels_train, axis=-1)

#images_test = np.expand_dims(images_test, axis=-1)
print(images_train.shape,images_validation.shape)
print(labels_train.shape ,labels_validation.shape)

(10000, 32, 32, 3) (10000, 32, 32, 3)
(10000, 10) (10000, 10)


In [5]:
# dataset normalization
# https://gruuuuu.github.io/machine-learning/cifar10-cnn/
print ("mean before normalization:", np.mean(images_train)) 
print ("std before normalization:", np.std(images_train))
mean=[0,0,0]
std=[0,0,0]
newX_train  = np.ones(images_train.shape)
newX_test  = np.ones(images_validation.shape)
#train set에 있는 데이터로만 평균과 표준편차를 구함
for i in range(3):
    mean[i] = np.mean(images_train[:,:,:,i])
    std[i] = np.std(images_train[:,:,:,i])

#train과 test셋 모두 정규화 작업    
for i in range(3):
    newX_train[:,:,:,i] = images_train[:,:,:,i] - mean[i]
    newX_train[:,:,:,i] = newX_train[:,:,:,i] / std[i]
    newX_test[:,:,:,i] = images_validation[:,:,:,i] - mean[i]
    newX_test[:,:,:,i] = newX_test[:,:,:,i] / std[i]
        
images_train = newX_train
images_validation = newX_test

print ("mean after normalization:", np.mean(images_train))
print ("std after normalization:", np.std(images_train))
print(images_train.max())

mean before normalization: 121.04103444010417
std before normalization: 64.39060047846118
mean after normalization: 1.1627735811240806e-17
std after normalization: 1.0000000000000002
2.1122496548364804


In [11]:
vgg16 = tf.keras.applications.VGG16(weights=None,input_shape=(32,32,3),classes=10)
vgg16.compile(optimizer=tf.keras.optimizers.Adam(lr=0.005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
#vgg16.summary()
images_train.shape

(10000, 32, 32, 3)

In [13]:
#!mkdir cifar_vgg16_model
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=30),
             tf.keras.callbacks.ModelCheckpoint(filepath='./cifar_vgg16_model/best_model.h5',moniter='val_loss',save_best_only=True)]

In [14]:
history = vgg16.fit(images_train,labels_train, 
                    batch_size=128,epochs=100,
                    validation_data=(images_validation,labels_validation),
                    callbacks=callbacks)


Epoch 1/100
79/79 [==============================] - 4s 53ms/step - loss: 2.8385 - accuracy: 0.0996 - val_loss: 2.3036 - val_accuracy: 0.1025
Epoch 2/100
79/79 [==============================] - 3s 32ms/step - loss: 2.3036 - accuracy: 0.0941 - val_loss: 2.3043 - val_accuracy: 0.1016
Epoch 3/100
79/79 [==============================] - 4s 47ms/step - loss: 2.3029 - accuracy: 0.1020 - val_loss: 2.3032 - val_accuracy: 0.0952
Epoch 4/100
79/79 [==============================] - 3s 33ms/step - loss: 2.3028 - accuracy: 0.1017 - val_loss: 2.3034 - val_accuracy: 0.0952
Epoch 5/100
79/79 [==============================] - 3s 32ms/step - loss: 2.3026 - accuracy: 0.1012 - val_loss: 2.3033 - val_accuracy: 0.1016
Epoch 6/100
79/79 [==============================] - 3s 32ms/step - loss: 2.3027 - accuracy: 0.1013 - val_loss: 2.3033 - val_accuracy: 0.0980
Epoch 7/100
79/79 [==============================] - 4s 47ms/step - loss: 2.3027 - accuracy: 0.1030 - val_loss: 2.3032 - val_accuracy: 0.1003
Epoch 

KeyboardInterrupt: 

In [ ]:
print("ACC: ",history.history['accuracy'][0],"Val_ACC :",history.history['val_accuracy'][0])
print("LOSS: ",history.history['loss'][0],"Val_LOSS :",history.history['val_loss'][0])
